 ![CellphoneDB Logo](https://www.cellphonedb.org/images/cellphonedb_logo_33.png) | CellphoneDB is a publicly available repository of curated receptors, ligands and their interactions. ||
 :------------- | :------------- | :-------------

CellphoneDB includes subunit architecture for both ligands and receptors, representing heteromeric complexes accurately. This is crucial, as cell-cell communication relies on multi-subunit protein complexes that go beyond the binary representation used in most databases and studies.

CellPhoneDB integrates existing datasets that pertain to cellular communication and new manually reviewed information. CellPhoneDB utilises information from the following data bases: [UniProt](https://www.uniprot.org/), [Ensembl](https://www.ensembl.org/), [PDB](https://www.ebi.ac.uk/pdbe/), [the IMEx consortium](https://www.imexconsortium.org/) and [IUPHAR](https://www.guidetopharmacology.org/).

CellPhoneDB can be used to search for a particular ligand/receptor or interrogate your own single-cell transcriptomics data.



## Install CellphoneDB package

In [ ]:
%%capture
!pip install "git+https://github.com/ventolab/CellphoneDB.git@bare-essentials"

## List CellphoneDB data releases

In [ ]:
from IPython.display import HTML, display
from cellphonedb.utils import db_releases_utils
display(HTML(db_releases_utils.get_remote_database_versions_html()))

## Generate own input files
#### Note: use this section if need to append your own interactions to CellphoneDB's ones (or replace CellphoneDB's interactions with your own).

In [ ]:
import os
from cellphonedb.utils import generate_input_files
import warnings
warnings.filterwarnings('ignore')
user_dir_root = os.path.join(os.path.expanduser('~'),".cpdb")
user_files_dir = os.path.join(user_dir_root,"user_files")
cellophonedb_version = "v5.0.0"
# Use this file to add own interactions to 
path_to_user_interactions=os.path.join(user_files_dir,"test_interaction.csv")
path_to_user_complexes=os.path.join(user_files_dir,"test_complex.csv")
use_user_interactions_only=False
# The command below generates *_input.csv files in <user_dir_root>/<cellophonedb_version>/data/generated directory
generate_input_files.generate_all(user_dir_root, cellophonedb_version, \
    user_complex=path_to_user_complexes, user_interactions=path_to_user_interactions, user_interactions_only=False)

## Download input files from CellphoneDB
#### Note: use this section if you need to use interactions provided by CellphoneDB only.

In [ ]:
# The command below downloads from https://github.com/ventolab/cellphonedb-data/ *_input.csv files into
# <user_dir_root>/<cellophonedb_version>/data
from cellphonedb.utils import db_utils
cellophonedb_version = "v5.0.0"
db_utils.download_input_files(user_dir_root, cellophonedb_version)

## Load CellphoneDB into memory
#### Create DB file from input files created or downloaded above and load DB into memory.

In [ ]:
import os
from cellphonedb.utils import db_utils
# Specify paths to *_input.csv files to be used to generate the DB file
user_dir_root = os.path.join(os.path.expanduser('~'),".cpdb")
cellophonedb_version = "v5.0.0"
db_dir = db_utils.get_db_path(user_dir_root, cellophonedb_version)
data_dir = os.path.join(db_dir, "data")
# Use the following if you generated you own *_input.csv files
# data_dir = os.path.join(db_dir, "data","generated")
gene_input_path = os.path.join(data_dir, "gene_input.csv")
protein_input_path = os.path.join(data_dir, "protein_input.csv")
complex_input_path = os.path.join(data_dir, "complex_input.csv")
interaction_input_path = os.path.join(data_dir, "interaction_input.csv")
# Create <user_dir_root>/releases/<cellophonedb_version>/data/cellphonedb.zip using the input files above        
db_utils.create_db(user_dir_root, cellophonedb_version, \
    gene_input=gene_input_path, protein_input=protein_input_path, \
    complex_input=complex_input_path, interaction_input=interaction_input_path)
# Load into memory CellphoneDB data
interactions, genes, complex_composition, complex_expanded = \
    db_utils.get_interactions_genes_complex(user_dir_root, cellophonedb_version)

## Search CellphoneDB Interactions
#### Search CellphoneDB interactions by (a comma- or space-separated list of): 
* Ensembl ID (e.g. ENSG00000165029), 
* Gene name (e.g. ABCA1), 
* UniProt ID (e.g. KLRG2_HUMAN), 
* UniProt Accession (e.g. A4D1S0) or 
* Complex name (e.g. 12oxoLeukotrieneB4_byPTGR1)

In [ ]:
import os
from cellphonedb.utils import utils, search_utils
from IPython.display import HTML, display
# Search CellphoneDB interactions by (a comma- or space-separated list of):
# Ensembl ID (e.g. ENSG00000165029), Gene name (e.g. ABCA1), UniProt ID (e.g. KLRG2_HUMAN), 
# UniProt Accession (e.g. A4D1S0) or Complex name (e.g. 12oxoLeukotrieneB4_byPTGR1)
(results, complex_name2proteins_text) = search_utils.search('ENSG00000134780,integrin_a10b1_complex', user_dir_root, cellophonedb_version)
# Display results in a html table
# Note: Mouse over complex names to see constituent proteins
display(HTML(search_utils.get_html_table(results, complex_name2proteins_text)))
# Optionally, save results_csv to a file
# utils.write_to_csv(results, os.path.join(user_dir_root,"cpdb_search_results.csv"), delimiter=',')

## Load User Files
#### Note: User files are expected to be found in directory: \<user_dir_root\>/user_files 

In [ ]:
from cellphonedb.src.core.preprocessors import method_preprocessors
from cellphonedb import controller
#counts, raw_meta, meta, microenvs, degs = controller.get_user_files(user_dir_root, \
#    counts_fn='test_counts.txt', meta_fn='test_meta.txt', \
#    microenvs_fn='test_microenviroments.txt', degs_fn='test_degs.txt')
adata, counts, raw_meta, meta, microenvs, degs = controller.get_user_file(user_dir_root, h5ad_fn = 'test.h5ad')
meta = method_preprocessors.meta_preprocessor(raw_meta)
counts = controller._counts_validations(counts, meta)
subsampler = None
if subsampler is not None:
    counts = subsampler.subsample(counts)

## Run Basic Analysis

In [ ]:
from cellphonedb.src.core.methods import cpdb_analysis_method
means, significant_means, deconvoluted = cpdb_analysis_method.call(
    meta,
    counts,
    'ensembl',
    interactions,
    genes,
    complex_expanded,
    complex_composition,
    microenvs=microenvs,
    debug=False,
    output_path=os.path.join(user_dir_root,'out'))
# print(means.info)
# print(significant_means.info)
# print(deconvoluted.info)

## Run Statistical Analysis

In [ ]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
deconvoluted, means, pvalues, significant_means = \
    cpdb_statistical_analysis_method.call(meta,
        counts,
        'ensembl',
        interactions,
        genes,
        complex_expanded,
        complex_composition,
        microenvs=microenvs,
        iterations = 1000,
        threshold = 0.1,
        threads = 4,
        debug_seed = -1,
        result_precision = 3,
        pvalue = 0.05,
        separator = '|',
        debug = False,
        output_path = '')
# print(deconvoluted.info)
# print(means.info)
# print(pvalues.info)
# print(significant_means.info)


## Run Differential Analysis

In [ ]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method
deconvoluted, means, relevant_interactions, significant_means = \
    cpdb_degs_analysis_method.call(meta,
        counts,
        degs,
        'ensembl',
        interactions,
        genes,
        complex_expanded,
        complex_composition,
        microenvs=microenvs,
        iterations = 1000,
        threshold = 0.1,
        threads = 4,
        debug_seed = -1,
        result_precision = 3,
        separator = '|',
        debug = False,
        output_path = '')
# print(deconvoluted.info)
# print(means.info)
# print(relevant_interactions.info)
# print(significant_means.info)

## Plot Statistical Analysis results

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ktplotspy.plot import plot_cpdb, plot_cpdb_heatmap

# Exmaple dot plot
g1 = plot_cpdb(
        adata=adata,
        cell_type1="Myeloid",
        # '.' means any cell type
        cell_type2=".",
        means=means,
        pvals=pvalues,
        celltype_key="cell_type",
        genes=["FN1", "integrin-a5b1-complex","COLEC12"],
        title="Example dot plot"
    )

# Example heatmap
g2 = plot_cpdb_heatmap(
        adata=adata,
        pvals=pvalues,
        celltype_key="cell_type",
        log1p_transform=True,
        title="Example heatmap"
    )
g1, g2
